In [1]:
import parmed as pmd
import numpy as np
import os
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%matplotlib inline

/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
# Use ParmeEd to load in `prmtop` and write `frcmod`.
# Then, extract just the dihedral section...
# (Maybe I should also include the impropers...)
# sed -n '/DIH/,/IMP/p' hg.frcmod > hg.dih.csv
# http://ambermd.org/formats.html#frcmod

smirnoff = pd.read_csv("systems/a-bam-p/smirnoff/hg.dih.csv",
           header=1, skipfooter=1, sep="\s{2,}",
           names=["atoms", "?", "phi", "phase", "per", "scaling"])

1. Limit this to `:MGO`
2. Find a way to write this by atom name...? Perhaps not possible.
3. Loop through dihedrals in ParmEd, write them in a similar format to `frcmod` and see if I can spot the differences.

In [302]:
smirnoff.drop_duplicates(subset=["phi", "phase", "per"], inplace=True)

In [303]:
len(smirnoff)

18

In [298]:
bgbg_tip3p = pd.read_csv("systems/a-bam-p/bgbg-tip3p/hg.dih.csv",
           header=1, skipfooter=1, sep="\s{2,}",
           names=["atoms", "?", "phi", "phase", "per", "scaling"])

/home/dslochower/data/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [306]:
bgbg_tip3p.drop_duplicates(subset=["phi", "phase", "per"], inplace=True)

In [307]:
len(bgbg_tip3p)

17

In [314]:
smirnoff.sort_values(by="phi", ascending=False)

,atoms,?,phi,phase,per,scaling
191,16-1 -3 -33,1,1.350000,180.0,1.0,SCEE=1.0 SCNB=1.0
0,3 -1 -4 -6,1,1.175000,0.0,2.0,SCEE=1.0 SCNB=1.0
190,16-1 -3 -33,1,0.850000,180.0,-2.0,SCEE=1.0 SCNB=1.0
643,1 -3 -33-34,1,0.383333,0.0,3.0,SCEE=1.20000048 SCNB=2.0
183,1 -3 -33-35,1,0.383000,0.0,-3.0,SCEE=1.20000048 SCNB=2.0
10,1 -4 -8 -12,1,0.250000,180.0,-2.0,SCEE=1.0 SCNB=1.0
292,3 -1 -4 -5,1,0.250000,0.0,1.0,SCEE=1.0 SCNB=1.0
11,1 -4 -8 -12,1,0.200000,180.0,1.0,SCEE=1.0 SCNB=1.0
9,1 -4 -8 -12,1,0.180000,0.0,-3.0,SCEE=1.20000048 SCNB=2.0
563,5 -4 -6 -7,1,0.166667,0.0,3.0,SCEE=1.20000048 SCNB=2.0


In [315]:
bgbg_tip3p.sort_values(by="phi", ascending=False)

,atoms,?,phi,phase,per,scaling
0,os-c3-os-c3,1,1.350000,180.0,-1.0,SCEE=1.2 SCNB=2.0
3,os-c3-c3-os,1,1.175000,0.0,-2.0,SCEE=1.2 SCNB=2.0
1,os-c3-os-c3,1,0.850000,180.0,-2.0,SCEE=1.2 SCNB=2.0
25,h1-c3-os-c3,1,0.383333,0.0,3.0,SCEE=1.2 SCNB=2.0
8,c3-c3-os-c3,1,0.383000,0.0,3.0,SCEE=1.2 SCNB=2.0
17,os-c3-c3-h1,1,0.250000,0.0,-1.0,SCEE=1.2 SCNB=2.0
11,c3-c3-c3-c3,1,0.250000,180.0,-2.0,SCEE=1.2 SCNB=2.0
10,c3-c3-c3-c3,1,0.200000,180.0,-1.0,SCEE=1.2 SCNB=2.0
12,c3-c3-c3-c3,1,0.180000,0.0,3.0,SCEE=1.2 SCNB=2.0
16,h1-c3-oh-ho,1,0.166667,0.0,3.0,SCEE=1.2 SCNB=2.0


In [319]:
smirnoff_dihedrals = find_dihedrals(smirnoff)
bgbg_tip3p_dihedrals = find_dihedrals(bgbg_tip3p)

In [320]:
smirnoff_dihedrals

,atom1,atom2,atom3,atom4,phi_k,per,phase
0,O1,C1,C2,O2,0.144000,3,0.0
1,O1,C1,C2,O2,1.175000,2,0.0
2,O1,C1,C2,C3,0.156000,3,0.0
3,O5,C1,C2,O2,0.144000,3,0.0
4,O5,C1,C2,O2,1.175000,2,0.0
5,O5,C1,C2,C3,0.156000,3,0.0
6,C1,C2,C3,O3,0.156000,3,0.0
7,C1,C2,C3,C4,0.180000,3,0.0
8,C1,C2,C3,C4,0.250000,2,180.0
9,C1,C2,C3,C4,0.200000,1,180.0


In [321]:
bgbg_tip3p_dihedrals

,atom1,atom2,atom3,atom4,phi_k,per,phase
0,O5,C1,O1,C4,1.350000,1,180.000077
1,O5,C1,O1,C4,0.850000,2,180.000077
2,O5,C1,O1,C4,0.100000,3,0.000000
3,C3,C2,C1,O5,0.155556,3,0.000000
4,O2,C2,C1,O5,1.175000,2,0.000000
5,O5,C1,C2,O2,0.144000,3,0.000000
6,C2,C1,O1,C4,0.100000,2,180.000077
7,C2,C1,O1,C4,0.383000,3,0.000000
8,C2,C1,O5,C5,0.100000,2,180.000077
9,C2,C1,O5,C5,0.383000,3,0.000000


In [336]:
smirnoff_torsion_count = smirnoff_dihedrals.groupby(["atom1", "atom2", "atom3", "atom4"]).size().reset_index().rename(columns={0:'count'})


In [337]:
bgbg_torsion_count = bgbg_tip3p_dihedrals.groupby(["atom1", "atom2", "atom3", "atom4"]).size().reset_index().rename(columns={0:'count'})


In [356]:
smirnoff_torsion_count.head()
smirnoff_torsion_count["count"] = smirnoff_torsion_count["count"].astype(int)

In [361]:
bgbg_torsion_count.head()
bgbg_torsion_count["count"] = bgbg_torsion_count["count"].astype(int)

In [364]:
merged_torsions = pd.merge(smirnoff_torsion_count, bgbg_torsion_count, 
                           on=["atom1", "atom2", "atom3", "atom4"], 
                           how='outer', 
                           suffixes=('_smirnoff', '_bgbg_tip3p'),)

In [366]:
merged_torsions["difference"] = merged_torsions["count_smirnoff"] - merged_torsions["count_bgbg_tip3p"]

In [367]:
merged_torsions[merged_torsions["difference"] != 0]

,atom1,atom2,atom3,atom4,count_smirnoff,count_bgbg_tip3p,difference
16,C3,C2,C1,O5,20,24.0,-4.0
32,H1,C1,C2,O2,48,24.0,24.0
35,H2,C2,C1,O5,40,44.0,-4.0
66,O2,C2,C1,O5,40,44.0,-4.0
73,O5,C1,C2,C3,4,NaN,NaN
74,O5,C1,C2,H2,8,4.0,4.0
75,O5,C1,C2,O2,8,4.0,4.0


# Parameter comparisons

I find there are a different number of dihedral parameters applied to a system parameterized with GAFF v1.7 and SMIRNOFF99Frosst.

# Conversion

I converted Amber parameters for alpha-cyclodextrin using this code...

Briefly,


2. Use MOL2 as input for a list of `OEMol`s.
2. Use PDB as topology.

3. Merge system by adding ParmEd `Structure`s.

# Method 1 (use ParmEd)

In [6]:
smirnoff_prmtop = pmd.load_file("systems/a-bam-p/smirnoff/a000/hg.prmtop")
smirnoff_prmtop = smirnoff_prmtop[":MGO"]

gaff_prmtop = pmd.load_file("systems/a-bam-p/bgbg-tip3p/hg.topo")
gaff_prmtop = gaff_prmtop[":MGO"]

In [11]:
print(f"SMIRNOFF99Frosst: {len(smirnoff_prmtop.dihedrals)} dihedrals and {len(smirnoff_prmtop.impropers)} impropers.")

SMIRNOFF99Frosst: 630 dihedrals and 0 impropers.


In [12]:
print(f"GAFF v1.7: {len(gaff_prmtop.dihedrals)} dihedrals and {len(gaff_prmtop.impropers)} impropers.")

GAFF v1.7: 624 dihedrals and 0 impropers.


# Method 2 (use `frcmod`)

Create a `frcmod` file using 

In [10]:
smirnoff = pd.read_csv("systems/a-bam-p/smirnoff/hg.dih.csv",
           header=1, skipfooter=1, sep="\s{2,}",
           names=["atoms", "div", "phi", "phase", "per", "scaling"])

/home/dslochower/data/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


In [13]:
len(smirnoff)

667

# Further information, using `label_molecule.py`